In [ ]:
import itertools

import matplotlib.pyplot as plt
import numpy as np

from stats import *
from utils import *
import visual as vi
from Inlier_Thresholder import Inlier_Thresholder
import scipy.io as spi
from time import time
import os
import seaborn as sns
from itertools import chain

In [ ]:
directory = 'C:/Users/carlo/IACV PROJECT/adelH'

# List all files in the directory
files = os.listdir(directory)

# Filter out only the .mat files
names = [file for file in files if file.endswith('.mat')]

# Load each .mat file
mat_data = {}
for file in names:
    file_path = os.path.join(directory, file)
    mat_data[file] = spi.loadmat(file_path)
    

# 1) LMEDS -> INLIER THRESHOLDS

In [ ]:
# input parameter, how many files to analyse
number_of_data_to_analyse=len(names)


thresholds = [[] for i in range(number_of_data_to_analyse)]
res_matrices=[]
part_matrices=[]
soft_matrices=[]

H_lmeds=[]



for i in range(number_of_data_to_analyse):
    data=mat_data[names[i]]
    res_mat=build_residual_matrix(data,verbose=False)
    
    res_matrices.append(res_mat)
    part_matrices.append(build_partition_matrix(res_mat))
    
    res=compute_inliers_residual_curve(data)
    
    data=mat_data[names[i]]
    
    img1, img2 = data["img1"], data["img2"]

    outliers, models = vi.group_models(data)["outliers"], vi.group_models(data)["models"]
    
    points=extract_points(models,data)
    
    labels = points[3]
    
    points=points[0]
    
    H_models=[]
    
    for m in range(len(models)):
        src = points["src_points"][m]
        dst = points["dst_points"][m]
        
        H,_=verify_LMEDS_H(src,dst,verbose=False)
        
        H_models.append(H)
        
    H_lmeds.append(H_models)
    #print(len(res))

    for j in range(len(res)):
        anomaly_detector=Inlier_Thresholder(res[j])

        labels, threshold=anomaly_detector.use_best_method(verbose=False)  # use best method among the statistical ones, best according to silhouette
        #print("eccolo: ", threshold)
        thresholds[i].append(threshold)
            
    #print(res_mat.shape)
    soft_matrices.append(soft_clustering_assignment(res_mat,thresholds[i]))



# 2.a) GC RANSAC WITH INLIER THRESHOLD

In [ ]:
# input parameter, how many files to analyse
number_of_data_to_analyse= len(names)



res_matrices=[]
part_matrices=[]
H_gc=[]

residuals=[]
soft_clus_assignment=[]
gc_masks=[]




for i in range(number_of_data_to_analyse):
    data=mat_data[names[i]]
    
    img1, img2 = data["img1"], data["img2"]

    outliers, models = vi.group_models(data)["outliers"], vi.group_models(data)["models"]
    
    points=extract_points(models,data)
    
    points=points[0]
    
    H_models=[]
    
    H_res_models=[]
    
    gc_masks_i=[]
    
    for m in range(len(models)):
        src = points["src_points"][m]
        dst = points["dst_points"][m]
        
        H,mask_i=verify_pygcransac_H(src,dst,img1,img2,threshold=thresholds[i][m],verbose=False)
        
        H_models.append(H)
        
        H_residual= residual_H1_wrt_H2(src,H,H_lmeds[i][m])
        
        H_res_models.append(H_residual)
        
        gc_masks_i.append(mask_i)
        
    gc_masks.append(gc_masks_i)
        
    residuals.append(H_res_models)
        
    H_gc.append(H_models)
    
    
    res=build_residual_matrix(data, plot=True, verbose=False, method="gc-ransac" , threshold=thresholds[i])
    part_mat=build_partition_matrix(res)
    labl=data["label"]
    mod_lab=np.where(labl>0)
    labl=labl[mod_lab].reshape(-1,1)
    labl=labl/np.max(labl)
    labl=np.abs(np.max(part_mat)-labl*np.max(part_mat))
    
    res_matrices.append(res)
    part_matrices.append(part_mat)
    plot_residual_matrix(part_mat,labl,title="a")
    
    soft_clustering=soft_clustering_assignment(res,thresholds[i])
    soft_clus_assignment.append(soft_clustering)
    

    

        
        

# 2.b) INFLUENCE FUNCTION

In [ ]:

## outlier scores for a single model of a single image
def outlier_score(src , dst):
    
    ## initial quantities
    N=src.shape[0]
    
    ## fit the baseline
    H,baseline_mask=cv2.findHomography(src, dst, method=0)#verify_pygcransac_H(src,dst,img1,img2, threshold, verbose=False)
    
    ## initialize the return quantity
    outlier_scores_H=np.zeros(N)
    
    
    ## loop through all the points
    for i in range(N):
        src_i = np.delete(src, i, axis=0)
        dst_i = np.delete(dst, i, axis=0)
        
        ## fit the model
        H_i,mask_i=cv2.findHomography(src_i, dst_i, method=0)#verify_pygcransac_H(src_i,dst_i,img1,img2, threshold, verbose=False)

        ## outlier score
        outlier_scores_H[i]=sum(sum((H-H_i)**2))
        
    ## return
    return outlier_scores_H

        


In [ ]:
# input parameter, how many files to analyse
number_of_data_to_analyse=len(names)

outlier_scores_H=[]

for i in range(number_of_data_to_analyse):
    data=mat_data[names[i]]
    
    img1, img2 = data["img1"], data["img2"]

    outliers, models = vi.group_models(data)["outliers"], vi.group_models(data)["models"]
    
    points=extract_points(models,data)
    
    tot_src=points[1]
    tot_dst=points[2]
    
    points=points[0]
    
    out_score_img_i_H=[]    
    
    for m in range(len(models)):
        
        
        src = points["src_points"][m]
        dst = points["dst_points"][m]
        

        out_score_img_i_H.append(outlier_score(src,dst))    
        
    outlier_scores_H.append(out_score_img_i_H)  
        
        

    

        
        

In [ ]:
for j in range(number_of_data_to_analyse):
    
    for i in range(len(outlier_scores_H[j])):
         print(f"img{j} model{i}")
         print(f"inliers avarage score ({sum(gc_masks[j][i])}): ",np.mean(outlier_scores_H[j][i][np.where(gc_masks[j][i]==1)]),f" Outliers avarage score ({len(gc_masks[j][i])-(sum(gc_masks[j][i]))}): " ,
               np.mean(outlier_scores_H[j][i][np.where(gc_masks[j][i]==0)]), f"Factor of the two: ", np.mean(outlier_scores_H[j][i][np.where(gc_masks[j][i]==0)])/np.mean(outlier_scores_H[j][i][np.where(gc_masks[j][i]==1)]))




In [ ]:
for j in range(number_of_data_to_analyse):
    
    for i in range(len(outlier_scores_H[j])):
        m=(outlier_scores_H[j][i]>=np.median(outlier_scores_H[j][i])+3.5*np.std(outlier_scores_H[j][i]))
        m=abs(m.astype(int)-1)

        data=mat_data[names[j]]
        im1=data["img1"]
        im2=data["img2"]

        outliers, models = vi.group_models(data)["outliers"], vi.group_models(data)["models"]

        points=extract_points(models,data)

        tot_src=points[1]
        tot_dst=points[2]

        points=points[0]

        src = points["src_points"][i][np.where(m==0)]
        dst = points["dst_points"][i][np.where(m==0)]
        plt.figure()
        draw_matches(im1,im2,src,dst)
        plt.show()

# 3) GC RANSAC ON INLIERS AND CORRECTIONS

In [ ]:
# input parameter, how many files to analyse
number_of_data_to_analyse= len(names)



res_matrices_2=[]
part_matrices_2=[]
H_gc_2=[]

residuals_2=[]
soft_clus_assignment_2=[]
gc_masks_2=[]

SRC_corrected=[]
DST_corrected=[]




for i in range(number_of_data_to_analyse):
    data=mat_data[names[i]]
    
    img1, img2 = data["img1"], data["img2"]

    outliers, models = vi.group_models(data)["outliers"], vi.group_models(data)["models"]
    
    points=extract_points(models,data)
    
    tot_src=points[1]
    
    tot_dst=points[2]
    
    points=points[0]
    
    H_models=[]
    
    H_res_models=[]
    
    gc_masks_i=[]
    
    num_of_inliers = np.sum(data["label"] != 0)
    print(num_of_inliers)
    
    residual_matrix = np.zeros((num_of_inliers, len(models)))
    
    SRC_corrected_img_i=[]
    DST_corrected_img_i=[]

    
    for m in range(len(models)):
        

        src = tot_src[np.where(soft_clus_assignment[i][:,m]==1)] #points["src_points"][m][np.where(gc_masks[i][m]==1)]
        dst = tot_dst[np.where(soft_clus_assignment[i][:,m]==1)] #points["dst_points"][m][np.where(gc_masks[i][m]==1)]
        
        src_1=points["src_points"][m]
        dst_1=points["dst_points"][m]
        
        H,mask_i=verify_pygcransac_H(src,dst,img1,img2,threshold=thresholds[i][m],verbose=False)
        
        H_models.append(H)
        
        H_residual= residual_H1_wrt_H2(src,H,H_lmeds[i][m])
        
        H_res_models.append(H_residual)
        
        gc_masks_i.append(mask_i)
        
        plt.figure()

        draw_matches(img1, img2, src_1, dst_1, matchColor=(255,0,0), mask=1 - gc_masks[i][m], H=H,show_correct=True)
        plt.show()
        
        residual_matrix[:, m] = compute_residual(tot_src, tot_dst, H)
        SRC_corrected_img_i.append(src)
        
        DST_corrected_img_i.append(projectiveTransform(src,H).reshape(src.shape[0],2))
        
    SRC_corrected.append(SRC_corrected_img_i)
    DST_corrected.append(DST_corrected_img_i)
        
    res_matrices_2.append(residual_matrix)
    
    gc_masks_2.append(gc_masks_i)
        
    residuals_2.append(H_res_models)
        
    H_gc_2.append(H_models)
    
    
    #res=build_residual_matrix(data, plot=True, verbose=False, method="gc-ransac" , threshold=thresholds[i])
    
    part_mat=build_partition_matrix(residual_matrix)
    labl=data["label"]
    mod_lab=np.where(labl>0)
    labl=labl[mod_lab].reshape(-1,1)
    labl=labl/np.max(labl)
    labl=np.abs(np.max(part_mat)-labl*np.max(part_mat))
    
    res_matrices_2.append(residual_matrix)
    part_matrices_2.append(part_mat)
    plot_residual_matrix(part_mat,labl,title="a")
    
    soft_clustering=soft_clustering_assignment(residual_matrix,thresholds[i])
    soft_clus_assignment_2.append(soft_clustering)
    

    

        
        

#### Corrections

In [ ]:
# input parameter, how many files to analyse
number_of_data_to_analyse=len(names)


for i in range(number_of_data_to_analyse):
    data=mat_data[names[i]]
    
    outliers, models = vi.group_models(data)["outliers"], vi.group_models(data)["models"]

    
    img1, img2 = data["img1"], data["img2"]
    
    for m in range(len(models)):
        src = SRC_corrected[i][m]
        dst = DST_corrected[i][m]
        
        draw_matches(img1, img2, src, dst, title="Corrections",matchColor=(255, 255, 0), singlePointColor=None, flags=2,
                 mask=None, H=None,show_correct=False)
